In [8]:
###################################### FILTRADO DE MONEDAS PARA ESTRATEGIA SANTAAAAAA
###################################################################################################################
import modulos as md
import numpy as np
#lista=['CELOUSDT']
lista=md.lista_de_monedas()
for symbol in lista:
    try:
        data = md.estrategia_santa(symbol, tp_flag = True)
        resultado = md.backtesting(data, plot_flag = False)
        if (resultado['Profit Factor']>1 or resultado['Return [%]'] >0) and resultado['Win Rate [%]']>50:
        #if resultado['Win Rate [%]']>50:
            print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)} - Win Rate [%]: {md.truncate(resultado['Win Rate [%]'],2)}")
    except Exception as falla:
        pass


EOSUSDT - Return [%]: 0.5% - # Trades: 3 - Profit Factor: 1.89 - Win Rate [%]: 66.66
ONTUSDT - Return [%]: 0.53% - # Trades: 4 - Profit Factor: 3.22 - Win Rate [%]: 75.0
BATUSDT - Return [%]: 0.04% - # Trades: 3 - Profit Factor: 1.18 - Win Rate [%]: 66.66
ALGOUSDT - Return [%]: 0.16% - # Trades: 3 - Profit Factor: 5.83 - Win Rate [%]: 66.66
OMGUSDT - Return [%]: 0.79% - # Trades: 5 - Profit Factor: 1.82 - Win Rate [%]: 60.0


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)
c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


SUSHIUSDT - Return [%]: 0.5% - # Trades: 7 - Profit Factor: 1.23 - Win Rate [%]: 57.14
SRMUSDT - Return [%]: 1.78% - # Trades: 20 - Profit Factor: 1.52 - Win Rate [%]: 75.0
FLMUSDT - Return [%]: 0.11% - # Trades: 3 - Profit Factor: 2.34 - Win Rate [%]: 66.66
MATICUSDT - Return [%]: 0.81% - # Trades: 7 - Profit Factor: 1.49 - Win Rate [%]: 57.14
OCEANUSDT - Return [%]: 0.31% - # Trades: 5 - Profit Factor: 1.38 - Win Rate [%]: 60.0
CVCUSDT - Return [%]: 0.38% - # Trades: 5 - Profit Factor: 5.82 - Win Rate [%]: 80.0
RVNUSDT - Return [%]: 0.03% - # Trades: 4 - Profit Factor: 2.08 - Win Rate [%]: 75.0
NKNUSDT - Return [%]: 0.11% - # Trades: 8 - Profit Factor: 1.34 - Win Rate [%]: 62.5
SCUSDT - Return [%]: 0.01% - # Trades: 8 - Profit Factor: 2.04 - Win Rate [%]: 62.5
BAKEUSDT - Return [%]: 0.09% - # Trades: 6 - Profit Factor: 1.39 - Win Rate [%]: 66.66
CELOUSDT - Return [%]: 1.32% - # Trades: 11 - Profit Factor: 1.23 - Win Rate [%]: 54.54
FLOWUSDT - Return [%]: -0.01% - # Trades: 9 - Profit

In [ ]:
###################################### FILTRADO DE MONEDAS PARA ESTRATEGIA BB
###################################################################################################################

import modulos as md
import numpy as np
#lista=['SRMUSDT']
lista=md.lista_de_monedas()
for symbol in lista:
    try:
        data = md.estrategia_bb(symbol, tp_flag = True)
        resultado = md.backtesting(data, plot_flag = False)
        if resultado['Profit Factor'] > 1 and resultado[6]/resultado[17] >=0.3:
            print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)}")
    except Exception as falla:
        pass